In [ ]:
!pip uninstall -y typing-extensions typing-inspection starlette pydantic grpcio fastapi ipython torch torchvision torchaudio

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

In [ ]:
!git clone https://github.com/ace-step/ACE-Step.git
%cd ACE-Step

In [ ]:
!pip install -r requirements.txt
!pip install -e .

In [ ]:
import sys
import os

repo_path = os.getenv('ACE_STEP_REPO_PATH', os.getcwd())
os.chdir(repo_path)

if repo_path not in sys.path:
    sys.path.append(repo_path)

print("Working directory:", os.getcwd())
print("In sys.path:", repo_path in sys.path)

In [ ]:
import os
from huggingface_hub import snapshot_download

repo_id = "ACE-Step/ACE-Step-v1-3.5B"
cache_root = os.getenv('ACE_STEP_WEIGHTS_PATH', os.path.join(repo_path, '..', 'acestep_checkpoints'))
os.makedirs(cache_root, exist_ok=True)

snapshot_dir = snapshot_download(repo_id=repo_id, cache_dir=cache_root)
print("Snapshot directory:", snapshot_dir)

In [ ]:
import torch
from acestep.pipeline_ace_step import ACEStepPipeline

use_cuda = torch.cuda.is_available()
device_id = 0 if use_cuda else -1
use_bf16 = True

dtype = torch.bfloat16 if (use_bf16 and use_cuda) else torch.float32

pipe = ACEStepPipeline(
    checkpoint_dir=snapshot_dir,
    dtype=dtype,      
    device_id=device_id,           
    torch_compile=False
)

pipe.load_checkpoint(checkpoint_dir=snapshot_dir)
print("Pipeline loaded.")

In [ ]:
import os

PROMPT_FILE = "prompt_input.txt"
LYRICS_FILE = "lyrics_input.txt"

def _read(path: str) -> str:
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read().strip()
    except FileNotFoundError:
        print(f"[warn] Missing {path}; using empty text")
        return ""
    except UnicodeDecodeError:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read().strip()

prompt_text = _read(PROMPT_FILE)
lyrics_text = _read(LYRICS_FILE)

print("Prompt chars:", len(prompt_text))
print("Lyrics chars:", len(lyrics_text))

In [ ]:
def generate_music(text_prompt: str, duration: int = 30):
    waveform = model.generate(text=text_prompt, duration=duration)
    return waveform

In [ ]:
duration = 60
audio_path = os.path.join(os.path.dirname(snapshot_dir), "generated_music.wav")

pipe(
    audio_duration=duration,
    prompt=prompt_text,
    lyrics=lyrics_text,
    infer_step=60,                
    guidance_scale=15.0,           
    scheduler_type="euler",       
    cfg_type="apg",                
    omega_scale=10.0,
    manual_seeds="",
    guidance_interval=0.5,
    guidance_interval_decay=0.0,
    min_guidance_scale=3.0,
    use_erg_tag=True,
    use_erg_lyric=True,
    use_erg_diffusion=True,
    oss_steps="",
    guidance_scale_text=0.0,
    guidance_scale_lyric=0.0,
    save_path=audio_path
)

print("Saved to:", audio_path)